In [1]:
import json
import pandas as pd
import numpy as np
#regular expression library
import re
#sql module
from sqlalchemy import create_engine

ModuleNotFoundError: No module named 'sqlalchemy'

In [ ]:
#create variable for the path
file_dir = "/Users/dbeliankou/Documents/Berkley Bootcamp/LocalRepo/Repo_Module_8/Movies-ETL"

In [ ]:
#f'{file_dir}filename'
#Read the JSON file
with open(f'{file_dir}/wikipedia.movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)

In [ ]:
len(wiki_movies_raw)

In [ ]:
# First 5 records
wiki_movies_raw[:5]

In [ ]:
# Last 5 records
wiki_movies_raw[-5:]

In [ ]:
# Some records in the middle
wiki_movies_raw[3600:3605]

In [ ]:
kaggle_metadata = pd.read_csv(f'{file_dir}/movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir}/ratings.csv')

In [ ]:
ratings.sample(5)

In [ ]:
wiki_movies_df = pd.DataFrame(wiki_movies_raw)
wiki_movies_df.head()

In [ ]:
wiki_movies_df.columns.tolist()

In [ ]:
#Filter the data only for those movies that dave a director and iMDB link, and no episodes(we need only movies)
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]
len(wiki_movies)

In [ ]:
#make a dF from wiki_movies
wiki_movies_df2 =pd.DataFrame(wiki_movies)
wiki_movies_df2.head()

In [ ]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    return movie

In [ ]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]

In [ ]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]['url']

In [ ]:
sorted(wiki_movies_df.columns.tolist())

In [ ]:
#Make an empty dict to hold all of the alternative titles.
#def clean_movie(movie):
#    movie = dict(movie) #create a non-destructive copy
#    alt_titles = {}
#Loop through a list of all alternative title keys.
#    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
#                'Hangul','Hebrew','Hepburn','Japanese','Literally',
#                'Mandarin','McCune–Reischauer','Original title','Polish',
#                'Revised Romanization','Romanized','Russian',
#                'Simplified','Traditional','Yiddish']:
#        #Check if the current key exists in the movie object.
#        if key in movie:
#            #If so, remove the key-value pair and add to the alternative titles dictionary.
#            alt_titles[key] = movie[key]
#            movie.pop(key)
#            #After looping through every key, add the alternative titles dict to the movie object.
#    if len(alt_titles) > 0:
#        movie['alt_titles'] = alt_titles

#    return movie

In [ ]:
#list of cleaned movies with a list comprehension
#clean_movies = [clean_movie(movie) for movie in wiki_movies]
#wiki_movies_df = pd.DataFrame(clean_movies)
#sorted(wiki_movies_df.columns.tolist())

In [ ]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    # combine alternate titles into one list
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    # merge other alternate column names 
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie

In [ ]:
clean_movies = [clean_movie(movie) for movie in wiki_movies]
wiki_movies_df = pd.DataFrame(clean_movies)
wiki_movies_df.head()

In [ ]:
#Drop duplicated iMDB links

wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wiki_movies_df))
wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
print(len(wiki_movies_df))
wiki_movies_df.head()

In [ ]:
#define the colums that we want to keep (more than 90% of values in columns is not null)
[column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]
wiki_movies_df.head()

In [ ]:
#CONVERT DATA TYPES
#Identify types
wiki_movies_df.dtypes

In [ ]:
#drop na in box office column
box_office = wiki_movies_df['Box office'].dropna() 

In [ ]:
#define non string values in box office
def is_not_a_string(x):
    return type(x) != str
box_office[box_office.map(is_not_a_string)]

In [ ]:
#use lambda function to find not a string
box_office[box_office.map(lambda x: type(x) != str)]

In [ ]:
#use join() as separator
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
#find patterns in box column
form_one = r'\$\d+\.?\d*\s*[mb]illion'
box_office.str.contains(form_one, flags=re.IGNORECASE).sum()

In [ ]:
#find pattern 2 in box column
form_two = r'\$\d{1,3}(?:,\d{3})+'
box_office.str.contains(form_two, flags=re.IGNORECASE).sum()

In [ ]:
#check if any values don't match any pattern
matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE)
matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE)
box_office[~matches_form_one & ~matches_form_two]


In [ ]:
#remove spaces btw $ and number
form_one = r'\$\s*\d+\.?\d*\s*[mb]illion'
form_two = r'\$\s*\d{1,3}(?:,\d{3})+'

In [ ]:
#either a comma or period as a thousands separator 
form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+'

In [ ]:
#reject if str “million” or “billion” after the number
form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'

In [ ]:
#repolace ranges
box_office = box_office.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

In [ ]:
#remove misspellings
form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'

In [ ]:
#extract box office info in df where values match form_1 OR form_2
box_office.str.extract(f'({form_one}|{form_two})')

In [ ]:
def parse_dollars(s):
    # if s is not a string, return NaN
    if type(s) != str:
        return np.nan

    # if input is of the form $###.# million
    if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " million"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a million
        value = float(s) * 10**6

        # return value
        return value

    # if input is of the form $###.# billion
    elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " billion"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a billion
        value = float(s) * 10**9

        # return value
        return value

    # if input is of the form $###,###,###
    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

        # remove dollar sign and commas
        s = re.sub('\$|,','', s)

        # convert to float
        value = float(s)

        # return value
        return value

    # otherwise, return NaN
    else:
        return np.nan

In [ ]:
#extract ans parse values from box office column
wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

In [ ]:
#drop raw box office column
wiki_movies_df.drop('Box office', axis=1, inplace=True)

In [ ]:
#start working on budget column
budget = wiki_movies_df['Budget'].dropna()
budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

In [ ]:
#parse budget column
matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE)
matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE)
budget[~matches_form_one & ~matches_form_two]

In [ ]:
#remove numbers in square brackets
budget = budget.str.replace(r'\[\d+\]\s*', '')
budget[~matches_form_one & ~matches_form_two]


In [ ]:
#remove non-parsed budget column
wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
wiki_movies_df.drop('Budget', axis=1, inplace=True)

In [ ]:
# PARSE RELEASE DATE
#column with non-null data
release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
#describe different pattern for date
date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
date_form_two = r'\d{4}.[01]\d.[123]\d'
date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
date_form_four = r'\d{4}'

In [ ]:
#extract dates
release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)

In [ ]:
#transform data to date-time format
wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)
wiki_movies_df['release_date']

In [ ]:
# PARSE RUNNING TIME
#column with non-null data
running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
# check how many running times look "... minutes"
running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE).sum()

In [ ]:
#inspect the rest
running_time[running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE) != True]

In [ ]:
# use other abbreviation of "minutes" to cut it to "m"
running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE).sum()

In [ ]:
#check the rest
running_time[running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE) != True]

In [ ]:
#relaxing the condition that the pattern has to start at the beginning of the string
running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')

In [ ]:
#convert data to numeric values, using coercing fill empty strings with NaN and then change NaN to 0 using coercing
running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)

In [ ]:
#convert hours to minutes
wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)

In [ ]:
#drop raw column
wiki_movies_df.drop('Running time', axis=1, inplace=True)

In [ ]:
##CLEAN KAGGLE DATA
#check types
kaggle_metadata.dtypes

In [ ]:
#check that "adult" column contains bool values
kaggle_metadata['adult'].value_counts()

In [ ]:
#look at the data in aduln column that is not bool
kaggle_metadata[~kaggle_metadata['adult'].isin(['True','False'])]

In [ ]:
# remove adult movies
kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns')

In [ ]:
#check video column for non-bool values
kaggle_metadata['video'].value_counts()

In [ ]:
#convert data type and assign it to video column
kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'


In [ ]:
#convert values to numeric data type
kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')

In [ ]:
#convert release column to date-time format
kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])

In [ ]:
#check if there null values in ratings
ratings.info(null_counts=True)

In [ ]:
#check converting timestamp to date_time with origin is 'unix' and the time unit is seconds.
pd.to_datetime(ratings['timestamp'], unit='s')

In [ ]:
# assighn format to timestamp format
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

In [ ]:
#check ratings for errors with histogram and describe function
ratings['rating'].plot(kind='hist')
ratings['rating'].describe()

In [ ]:
#merge two tables and identify which table each column came from
movies_df = pd.merge(wiki_movies_df, kaggle_metadata, on='imdb_id', suffixes=['_wiki','_kaggle'])

In [ ]:
# Competing data:
# Wiki                     Movielens                Resolution
#--------------------------------------------------------------------------
# title_wiki               title_kaggle             Drop Wiki
# running_time             runtime                  Keep Kaggle; fill in zeros with Wikipedia data.
# budget_wiki              budget_kaggle            Keep Kaggle; fill in zeros with Wikipedia data.
# box_office               revenue                  Keep Kaggle; fill in zeros with Wikipedia data.
# release_date_wiki        release_date_kaggle      Drop Wiki
# Language                 original_language        Drop Wiki
# Production company(s)    production_companies     Drop Wiki 

In [ ]:
#work with title
movies_df[['title_wiki','title_kaggle']]

In [ ]:
#Look at the rows where the titles don’t match.
movies_df[movies_df['title_wiki'] != movies_df['title_kaggle']][['title_wiki','title_kaggle']]

In [ ]:
#check missind values at the kaggle data
movies_df[(movies_df['title_kaggle'] == '') | (movies_df['title_kaggle'].isnull())]

In [ ]:
#work with runtime
#fill missing values with zero
movies_df.fillna(0).plot(x='running_time', y='runtime', kind='scatter')

In [ ]:
#work with budget
movies_df.fillna(0).plot(x='budget_wiki',y='budget_kaggle', kind='scatter')


In [ ]:
#work with box_office
movies_df.fillna(0).plot(x='box_office', y='revenue', kind='scatter')

In [ ]:
#scatterplot for movies less than 1 billion in the box office
movies_df.fillna(0)[movies_df['box_office'] < 10**9].plot(x='box_office', y='revenue', kind='scatter')

In [ ]:
#release date
movies_df[['release_date_wiki','release_date_kaggle']].plot(x='release_date_wiki', y='release_date_kaggle', style='.')

In [ ]:
#check the outliers
movies_df[(movies_df['release_date_wiki'] > '1996-01-01') & (movies_df['release_date_kaggle'] < '1965-01-01')]

In [ ]:
#drop outlier
#find index of a row
movies_df[(movies_df['release_date_wiki'] > '1996-01-01') & (movies_df['release_date_kaggle'] < '1965-01-01')].index

In [ ]:
#drop raw
movies_df = movies_df.drop(movies_df[(movies_df['release_date_wiki'] > '1996-01-01') & (movies_df['release_date_kaggle'] < '1965-01-01')].index)

In [ ]:
#check null values
movies_df[movies_df['release_date_wiki'].isnull()]

In [ ]:
#Language
#count wiki languages
movies_df['Language'].value_counts()


In [ ]:
#convert lists of languages to tuples to count it
movies_df['Language'].apply(lambda x: tuple(x) if type(x) == list else x).value_counts(dropna=False)

In [ ]:
#count kaggle languages
movies_df['original_language'].value_counts(dropna=False)

In [ ]:
#Production Companies
#check the samples
movies_df[['Production company(s)','production_companies']]

In [ ]:
#CLEAN THE DATA FRAME
#drop unnesessary wiki columns
movies_df.drop(columns=['title_wiki','release_date_wiki','Language','Production company(s)'], inplace=True)

In [ ]:
#define function to fill missing data in kaggle columns with wiki and drop wiki columns
def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
    df[kaggle_column] = df.apply(
        lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column]
        , axis=1)
    df.drop(columns=wiki_column, inplace=True)

In [ ]:
#run the function for necessary columns
fill_missing_kaggle_data(movies_df, 'runtime', 'running_time')
fill_missing_kaggle_data(movies_df, 'budget_kaggle', 'budget_wiki')
fill_missing_kaggle_data(movies_df, 'revenue', 'box_office')
movies_df

In [ ]:
#check columns with one value
for col in movies_df.columns:
    lists_to_tuples = lambda x: tuple(x) if type(x) == list else x
    value_counts = movies_df[col].apply(lists_to_tuples).value_counts(dropna=False)
    num_values = len(value_counts)
    if num_values == 1:
        print(col)

In [ ]:
#drop column with one value
movies_df['video'].value_counts(dropna=False)

In [ ]:
#list comprehension
#lists_to_tuples = lambda x: tuple(x) if type(x) == list else x
#value_counts = movies_df[col].apply(lists_to_tuples).value_counts(dropna=False)
#num_values = len(value_counts)
#comprehension_list = [lists_to_tuples = lambda x: tuple(x) if type(x) == list else x value_counts = movies_df[col].apply(lists_to_tuples).value_counts(dropna=False) num_values = len(value_counts) for col in movies_df.columns if num_values == 1]
#print(comprehension_list)

In [ ]:
#reorder columns
movies_df = movies_df[['imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection','url','imdb_link',
                       'runtime','budget_kaggle','revenue','release_date_kaggle','popularity','vote_average','vote_count',
                       'genres','original_language','overview','spoken_languages','Country',
                       'production_companies','production_countries','Distributor',
                       'Producer(s)','Director','Starring','Cinematography','Editor(s)','Writer(s)','Composer(s)','Based on'
                      ]]

In [ ]:
#rename columns
movies_df.rename({'id':'kaggle_id',
                  'title_kaggle':'title',
                  'url':'wikipedia_url',
                  'budget_kaggle':'budget',
                  'release_date_kaggle':'release_date',
                  'Country':'country',
                  'Distributor':'distributor',
                  'Producer(s)':'producers',
                  'Director':'director',
                  'Starring':'starring',
                  'Cinematography':'cinematography',
                  'Editor(s)':'editors',
                  'Writer(s)':'writers',
                  'Composer(s)':'composers',
                  'Based on':'based_on'
                 }, axis='columns', inplace=True)

In [ ]:
# group the “movieId” and “rating” columns and take the count for each group
rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
                .rename({'userId':'count'}, axis=1) 

In [ ]:
#pivot the rating data
rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
                .rename({'userId':'count'}, axis=1) \
                .pivot(index='movieId',columns='rating', values='count')

In [ ]:
#rename the column
rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]

In [ ]:
#merge movies df and rating df
movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='kaggle_id', right_index=True, how='left')

In [ ]:
#replace missing values with zero
movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)

In [ ]:
movies_with_ratings_df.head()